In [5]:
import datetime as dt

import pandas as pd

from sqlalchemy import create_engine, desc, distinct, func, MetaData, select, Table

In [8]:
engine = create_engine('sqlite:///data.sqlite')
connection = engine.connect()
metadata = MetaData()

In [9]:
# Créer l'objet Table
rests = Table('restaurants', metadata, autoload=True, autoload_with=engine)

In [10]:
# Lire dix lignes dans la table restaurants et les mettre dans un DataFrame
df = pd.read_sql_table('restaurants', connection)
df.shape

(8406, 12)

In [79]:
# Exporter vers un csv
df[['ADRESSE', 'COM', 'CODPOS', 'LIBCOM']].to_csv('adresses.csv')

In [82]:
# Après avoir utilisé adresse.data.gouv.fr, lire le fichier de sortie
df_geo = pd.read_csv('adresses.geocoded.csv', sep=';')

In [83]:
df_geo

,Unnamed: 0,ADRESSE,COM,CODPOS,LIBCOM,latitude,longitude,result_label,result_score,result_type,result_id,result_housenumber,result_name,result_street,result_postcode,result_city,result_context,result_citycode
0,0,9 BD RAIMBALDI,06088,6000.0,NICE,43.707022,7.268444,9 Boulevard Raimbaldi 06000 Nice,0.55,housenumber,ADRNIVX_0000000261615497,9,Boulevard Raimbaldi,NaN,6000.0,Nice,"06, Alpes-Maritimes, Provence-Alpes-Côte d'Azur",06088
1,1,32 RUE DE LA DEMOCRATIE,66053,66190.0,COLLIOURE,42.523644,3.085808,Rue de la Démocratie 66190 Collioure,0.69,street,66053_0090_e9ab51,NaN,Rue de la Démocratie,NaN,66190.0,Collioure,"66, Pyrénées-Orientales, Occitanie (Languedoc-...",66053
2,2,RUE JEAN MOULIN,63236,63240.0,MONT DORE,45.573600,2.810492,2 Rue Jean Moulin 63240 Mont-Dore,0.77,housenumber,ADRNIVX_0000000278568431,2,Rue Jean Moulin,NaN,63240.0,Mont-Dore,"63, Puy-de-Dôme, Auvergne-Rhône-Alpes (Auvergne)",63236
3,3,LE FOURCAT,81105,81300.0,GRAULHET,43.742795,2.015465,Le Fourcat 81300 Graulhet,0.69,street,81105_B113_3204b8,NaN,Le Fourcat,NaN,81300.0,Graulhet,"81, Tarn, Occitanie (Midi-Pyrénées)",81105
4,4,4 RUE CARNOT,89025,89200.0,AVALLON,47.491328,3.911941,4 Rue Carnot 89200 Avallon,0.71,housenumber,ADRNIVX_0000000270586581,4,Rue Carnot,NaN,89200.0,Avallon,"89, Yonne, Bourgogne-Franche-Comté (Bourgogne)",89025
5,5,28 GR GRANDE RUE,71346,71420.0,PERRECY LES FORGES,46.614332,4.216109,Grande Rue 71420 Perrecy-les-Forges,0.65,street,71346_XXXX_c20c3e,NaN,Grande Rue,NaN,71420.0,Perrecy-les-Forges,"71, Saône-et-Loire, Bourgogne-Franche-Comté (B...",71346
6,6,61 RUE AUGUSTE RENOIR,78146,78400.0,CHATOU,48.902721,2.158468,61 Rue Auguste Renoir 78400 Chatou,0.75,housenumber,ADRNIVX_0000000269454383,61,Rue Auguste Renoir,NaN,78400.0,Chatou,"78, Yvelines, Île-de-France",78146
7,7,12 RUE DU GOH LANNO,56177,56330.0,PLUVIGNER,47.778239,-3.004362,12 Rue du Goh Lanno 56330 Pluvigner,0.75,housenumber,ADRNIVX_0000002006043584,12,Rue du Goh Lanno,NaN,56330.0,Pluvigner,"56, Morbihan, Bretagne",56177
8,8,8 RUE PASCAL DUPRAT,40279,40990.0,SAINT PAUL LES DAX,43.722436,-1.053574,8 Rue Pascal Duprat 40990 Saint-Paul-lès-Dax,0.77,housenumber,ADRNIVX_0000000262625200,8,Rue Pascal Duprat,NaN,40990.0,Saint-Paul-lès-Dax,"40, Landes, Nouvelle-Aquitaine (Aquitaine)",40279
9,9,6 RUE CHARLES LECOCQ,62498,62300.0,LENS,50.429165,2.801242,6 Rue Charles Lecocq 62300 Lens,0.73,housenumber,ADRNIVX_0000000310885438,6,Rue Charles Lecocq,NaN,62300.0,Lens,"62, Pas-de-Calais, Hauts-de-France (Nord-Pas-d...",62498


In [ ]:
def geoloc(df, columns, postcode):
    """Géo-localise les adresses présentes dans les colonnes d'un df"""
    from io import StringIO
    
    # Ecrire les données dans un csv
    file = StringIO()
    df[columns].to_csv(file, index=False)
    
    # Revenir au début du fichier
    file.seek(0)
    
    # Envoyer les adresses à l'API adresse.data.gouv.fr
    r = requests.post("https://api-adresse.data.gouv.fr/search/csv/",
                      timeout=600,
                      files={'data': file},
                      params={'postcode': 'BILLINGPOSTALCODE'})
    
    # Lire les données dans un DataFrame
    return pd.read_csv(StringIO(r.text))

In [11]:
connection.close()